# Movie Recommendation Engine-Su, Chun Wen- Model 3: using Deep Learning model (VGG19) to find similar posters

## Step 1: Set up the enviornment and packages

In [1]:
import pandas as pd
import numpy as np
import os
import requests
import json
from sklearn.metrics import mean_squared_error


## Step 2: Create the get_poster function to retrieve posters from the Movie Database

In [2]:
import requests
import json
# Get base url filepath structure. w185 corresponds to size of movie poster.
headers = {'Accept': 'application/json'}
payload = {'api_key': '121243b2af1966b0d85754dc2b3bcc25'} 
response = requests.get("http://api.themoviedb.org/3/configuration", params=payload, headers=headers)
response = json.loads(response.text)
base_url = response['images']['base_url'] + 'w185'

def get_poster(imdbid, base_url):
    # Get IMDB movie ID
    movie_id = "tt0" + str(imdbid) 
    
    # Query themoviedb.org API for movie poster path.
    movie_url = 'http://api.themoviedb.org/3/movie/{:}/images'.format(movie_id)
    headers = {'Accept': 'application/json'}
    payload = {'api_key': '121243b2af1966b0d85754dc2b3bcc25'} 
    response = requests.get(movie_url, params=payload, headers=headers)
    try:
        file_path = json.loads(response.text)['posters'][0]['file_path']
    except:
        file_path = ""
        
    return (base_url + file_path, imdbid)

## Step 3: Create a folder to store the movie posters

In [3]:
# save posters in a folder
poster_folder='posters_final_part3/'
if poster_folder.split('/')[0] in os.listdir('./'):
    print('Folder already exists')
else:
    os.mkdir('./'+poster_folder)

## Step 4: Retrieve movie posters from the Movie Database.com using IMDB IDs and pickle this dataframe

In [4]:
df_id = pd.read_csv('links.csv', sep=',')

idx_to_movie = {}
for row in df_id.itertuples():
    idx_to_movie[row[1]-1] = row[2]  
    
total_movies = 9000

movies = [0]*total_movies
for i in range(len(movies)):
    if i in idx_to_movie.keys() and len(str(idx_to_movie[i])) == 6:
        movies[i] = (idx_to_movie[i]) 
movies = filter(lambda imdb: imdb != 0, movies)
#total_movies  = len(movies)

URL = [0]*total_movies 
IMDB = [0]*total_movies 
URL_IMDB = {"url":[],"imdb":[]}
i = 0
for movie in movies:
    (URL[i], IMDB[i]) = get_poster(movie, base_url)
    if URL[i] != base_url+"":
        URL_IMDB["url"].append(URL[i])
        URL_IMDB["imdb"].append(IMDB[i])
    i += 1 
# URL = filter(lambda url: url != base_url+"", URL)
df = pd.DataFrame(data=URL_IMDB)
df.to_pickle('imdb_data_p3.pkl')
df.to_csv('imdb_data_p3.csv')

In [5]:
print (df)

        imdb                                                url
0     114709  http://image.tmdb.org/t/p/w185/uMZqKhT4YA6mqo2...
1     113497  http://image.tmdb.org/t/p/w185/vgpXmVaVyUL7GGi...
2     113228  http://image.tmdb.org/t/p/w185/6ksm1sjKMFLbO7U...
3     114885  http://image.tmdb.org/t/p/w185/eT79mN6LqeDXeLi...
4     113041  http://image.tmdb.org/t/p/w185/e64sOI48hQXyru7...
5     113277  http://image.tmdb.org/t/p/w185/zMyfPUelumio3ti...
6     114319  http://image.tmdb.org/t/p/w185/jQh15y5YB7bWz1N...
7     112302  http://image.tmdb.org/t/p/w185/sGO5Qa55p7wTu7F...
8     114576  http://image.tmdb.org/t/p/w185/eoWvKD60lT95Ss1...
9     113189  http://image.tmdb.org/t/p/w185/trtANqAEy9dxRCe...
10    112346  http://image.tmdb.org/t/p/w185/lymPNGLZgPHuqM2...
11    112896  http://image.tmdb.org/t/p/w185/xve4cgfYItnOhtz...
12    112453  http://image.tmdb.org/t/p/w185/gV5PCAVCPNxlOLF...
13    113987  http://image.tmdb.org/t/p/w185/cICkmCEiXRhvZmb...
14    112760  http://image.tmdb.org/t/p/

## Step 5: Put the posters in a folder with .jpg file extension

In [8]:
#df = pd.DataFrame(data=URL_IMDB)
total_movies = len(df)  

import urllib.request

poster_path = "posters_final_part3/"
for i in range(total_movies):
    urllib.request.urlretrieve(df.url[i], poster_path + str(i) + ".jpg")

## Step 6: Load posters into image array and import VGG19

### *Note: keras.preprocessing image function is broke if pillow is not installed with version 5.1 version.  You will get an error message, "Could not import PIL.Image. The use of `array_to_img` requires PIL"*

In [9]:
#poster_path = "posters_final/"
#df=pd.read_pickle('imdb_data.pkl')

In [6]:
print(df)

        imdb                                                url
0     114709  http://image.tmdb.org/t/p/w185/uMZqKhT4YA6mqo2...
1     113497  http://image.tmdb.org/t/p/w185/vgpXmVaVyUL7GGi...
2     113228  http://image.tmdb.org/t/p/w185/6ksm1sjKMFLbO7U...
3     114885  http://image.tmdb.org/t/p/w185/eT79mN6LqeDXeLi...
4     113041  http://image.tmdb.org/t/p/w185/e64sOI48hQXyru7...
5     113277  http://image.tmdb.org/t/p/w185/zMyfPUelumio3ti...
6     114319  http://image.tmdb.org/t/p/w185/jQh15y5YB7bWz1N...
7     112302  http://image.tmdb.org/t/p/w185/sGO5Qa55p7wTu7F...
8     114576  http://image.tmdb.org/t/p/w185/eoWvKD60lT95Ss1...
9     113189  http://image.tmdb.org/t/p/w185/trtANqAEy9dxRCe...
10    112346  http://image.tmdb.org/t/p/w185/lymPNGLZgPHuqM2...
11    112896  http://image.tmdb.org/t/p/w185/xve4cgfYItnOhtz...
12    112453  http://image.tmdb.org/t/p/w185/gV5PCAVCPNxlOLF...
13    113987  http://image.tmdb.org/t/p/w185/cICkmCEiXRhvZmb...
14    112760  http://image.tmdb.org/t/p/

## Step 7: Use Keras to import VGG19 model.  Preprocess these posters into 224x224 RGB images

In [11]:
from keras.applications import VGG19
from keras.applications.vgg19 import preprocess_input
from keras.preprocessing import image as kimage

total_movies = 1272
image = [0]*total_movies
x = [0]*total_movies
for i in range(total_movies):
    image[i] = kimage.load_img(poster_path + str(i) + ".jpg", target_size=(224, 224))
    x[i] = kimage.img_to_array(image[i])
    x[i] = np.expand_dims(x[i], axis=0)
    x[i] = preprocess_input(x[i])

In [13]:
print(image[1271])

<PIL.Image.Image image mode=RGB size=224x224 at 0x26C7FA76278>


## Step 8: Create a model using VGG16 to recognize all poster images, then save the trained model

In [14]:
model = VGG19(include_top=False, weights='imagenet')
model.save('movie_recommend_p3.h5')

80142336/80134624 [==============================] - 28s 0us/step


In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

## Step 9: Use the trained model to predict similiarity of all movie posters and store them in a matrix

In [16]:
pred = [0]*total_movies
pred_norm = [0]*total_movies
matrix_res = np.zeros([total_movies,25088])
for i in range(total_movies):
    pred[i] = model.predict(x[i]).ravel()
    matrix_res[i,:] = pred[i]  

## Step 10: Save the prediction matrix (804MB: Very big)

In [11]:
#np.savetxt('matrix_res.txt',matrix_res)

## Step 11: Create a cosine similarity

In [17]:
sim = matrix_res.dot(matrix_res.T)
norms = np.array([np.sqrt(np.diagonal(sim))])
sim = sim / norms / norms.T 
len(pred[0])

25088

In [18]:
sim

array([[ 1.        ,  0.15645922,  0.09114654, ...,  0.08432057,
         0.07697601,  0.11719675],
       [ 0.15645922,  1.        ,  0.15402243, ...,  0.11749183,
         0.11358793,  0.16216776],
       [ 0.09114654,  0.15402243,  1.        , ...,  0.1105859 ,
         0.11474437,  0.15246596],
       ..., 
       [ 0.08432057,  0.11749183,  0.1105859 , ...,  1.        ,
         0.09856786,  0.18339942],
       [ 0.07697601,  0.11358793,  0.11474437, ...,  0.09856786,
         1.        ,  0.13756006],
       [ 0.11719675,  0.16216776,  0.15246596, ...,  0.18339942,
         0.13756006,  1.        ]])

## Step 12: Load in Movie data

In [19]:
# Load in movie data
idx_to_movie2 = {}
i = 0
for row in df.itertuples():
    idx_to_movie2[i] = row[1]
    i += 1  

In [20]:
print(idx_to_movie2)

{0: 114709, 1: 113497, 2: 113228, 3: 114885, 4: 113041, 5: 113277, 6: 114319, 7: 112302, 8: 114576, 9: 113189, 10: 112346, 11: 112896, 12: 112453, 13: 113987, 14: 112760, 15: 112641, 16: 114388, 17: 113101, 18: 112281, 19: 113845, 20: 113161, 21: 112722, 22: 112401, 23: 114168, 24: 113627, 25: 114057, 26: 114011, 27: 114117, 28: 112682, 29: 115012, 30: 112792, 31: 114746, 32: 112431, 33: 112637, 34: 112818, 35: 112286, 36: 113442, 37: 112697, 38: 112749, 39: 114279, 40: 112646, 41: 112573, 42: 113326, 43: 115645, 44: 113774, 45: 112373, 46: 115033, 47: 116606, 48: 114536, 49: 106473, 50: 103859, 51: 109424, 52: 114808, 53: 116324, 54: 110647, 55: 114131, 56: 112364, 57: 113107, 58: 113451, 59: 116130, 60: 113756, 61: 118055, 62: 115685, 63: 112585, 64: 112442, 65: 112342, 66: 112461, 67: 112427, 68: 109093, 69: 112384, 70: 114287, 71: 112288, 72: 112462, 73: 112495, 74: 112541, 75: 109370, 76: 112642, 77: 112688, 78: 112715, 79: 112740, 80: 109642, 81: 109686, 82: 111797, 83: 109759, 8

## Step 13: Find the top similar movie posters

In [21]:
def top_k_movies(similarity, mapper, movie_idx, k=6):
    return [mapper[x] for x in np.argsort(similarity[movie_idx,:])[:-k-1:-1]]

idx = 4
movies = top_k_movies(sim, idx_to_movie2, idx)
movies = movies[:5]

In [22]:
import requests
import json

from IPython.display import Image
from IPython.display import display
from IPython.display import HTML

n_display = 5
URL = [0]*n_display
i = 0
for movie in movies:
    (URL[i], IMDB[i]) = get_poster(movie, base_url)
    i += 1 
    
images = ''
for i in range(n_display):
    images += "<img style='width: 110px; margin: 0px; \
                float: left; border: 1px solid black;' src='%s' />" \
                % URL[i]

display(HTML(images))  

## Step 14: Try another poster

In [23]:
idx = 3
movies = top_k_movies(sim, idx_to_movie2, idx)
movies = movies[:5]

In [24]:
import requests
import json

from IPython.display import Image
from IPython.display import display
from IPython.display import HTML

n_display = 5
URL = [0]*n_display
i = 0
for movie in movies:
    (URL[i], IMDB[i]) = get_poster(movie, base_url)
    i += 1 
    
images = ''
for i in range(n_display):
    images += "<img style='width: 110px; margin: 0px; \
                float: left; border: 1px solid black;' src='%s' />" \
                % URL[i]

display(HTML(images))  

# End of Model 3